## 样本增强

In [6]:
#引入样本增强库
from keras.preprocessing.image import ImageDataGenerator

In [7]:
#创建样本生成器参数
datagen = ImageDataGenerator(
    rotation_range = 10,        #旋转
    width_shift_range = 0.08,   #横向移动
    height_shift_range = 0.08,  #纵向移动
    shear_range = 0.1,          #一虎毒逆时针方向剪切角度
    zoom_range = 0.1,           #随机缩放范围
    horizontal_flip = True,     #随机水平翻转
)

In [8]:
#生成器基本配置
gen_data = datagen.flow_from_directory(
    directory = '../data',        #读取目录  #类别子文件夹的上一级文件夹
    save_prefix = 'data',      #文件开头名字
    save_format = 'jpg',
    save_to_dir = '../data/day07/new',       #存入目录
    target_size = (100,100)    #目标尺寸
)

Found 6 images belonging to 1 classes.


In [38]:
#调用生成器
for i in range(5):
    gen_data.next()

Found 1 images belonging to 1 classes.


## VGG16

In [3]:
from keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dropout, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from keras.losses import categorical_crossentropy
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
import numpy as np
import os
from PIL import Image

In [4]:
#搭建方式一 ： 直接盖楼方式
inputs = Input(shape=(224,224,3))

#block1
x = Conv2D(64, (3, 3), padding='same', activation='relu')(inputs)
x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
#block2
x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
#block3
x = Conv2D(256, (3, 3), padding='same', activation='relu')(x)
x = Conv2D(256, (3, 3), padding='same', activation='relu')(x)
x = Conv2D(256, (3, 3), padding='same', activation='relu')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
#block4
x = Conv2D(512, (3, 3), padding='same', activation='relu')(x)
x = Conv2D(512, (3, 3), padding='same', activation='relu')(x)
x = Conv2D(512, (3, 3), padding='same', activation='relu')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
#block5
x = Conv2D(512, (3, 3), padding='same', activation='relu')(x)
x = Conv2D(512, (3, 3), padding='same', activation='relu')(x)
x = Conv2D(512, (3, 3), padding='same', activation='relu')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
#block6
x = Flatten()(x)
x = Dense(4096, activation='relu')(x)
x = Dense(4096, activation='relu')(x)
outputs = Dense(1000, activation='softmax')(x)
model = Model(inputs = inputs, outputs = outputs)
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 128)       0     

In [5]:
#枚举+循环
def vgg16():
    inputs = Input(shape=(224, 224, 3)) 
    x = inputs  #x和 input要分离，要循环x，inputs是第一层
    blocks = [2,2,3,3,3] #枚举list的每个位置，即index键代表第几个block，值（value）代表block里的conv的数量
    for block, convs in enumerate(blocks):
        if block == 4:
            for conv in range(convs):
                x = Conv2D(512, (3,3), padding='same', activation='relu')(x)
            x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
        else:
            for conv in range(convs):
                x = Conv2D(64 * 2 ** block, (3,3), padding='same', activation='relu')(x)
            x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dense(4096, activation='relu')(x)
    outputs = Dense(1000, activation='softmax')(x)
    model = Model(inputs = inputs, outputs = outputs)
    return model
vgg16().summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 56, 56, 128)       0   

SyntaxError: invalid syntax (CreatorTemp/ipykernel_22840/286204653.py, line 1)